<a href="https://colab.research.google.com/github/shivammehta007/NLPResearch/blob/master/Tutorials/Natural%20Language%20Processing/PyTorch%20Sentimental%20Analysis/First_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch 
from torchtext import data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
SEED = 1234
torch.manual_seed(SEED)

In [0]:
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype=torch.float)

In [4]:
from torchtext import datasets
train_dataset , test_dataset = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:08<00:00, 10.5MB/s]


In [5]:
len(train_dataset), len(test_dataset)

(25000, 25000)

In [0]:
import random

train_data, validation_data = train_dataset.split(random_state= random.seed(SEED))

In [7]:
len(train_data), len(validation_data)

(17500, 7500)

In [0]:
MAX_VOCAB_SIZE = 25000
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [9]:
print(len(TEXT.vocab), len(LABEL.vocab))
print(TEXT.vocab.freqs.most_common(20))


25002 2
[('the', 202273), (',', 191724), ('.', 165509), ('and', 109826), ('a', 109039), ('of', 100566), ('to', 93435), ('is', 76237), ('in', 61513), ('I', 53946), ('it', 53635), ('that', 49166), ('"', 44415), ("'s", 42942), ('this', 42362), ('-', 37016), ('/><br', 35354), ('was', 35200), ('as', 30258), ('with', 29916)]


In [30]:
torch.cuda.is_available()

True

In [0]:
BATCH_SIZES= 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits( (train_data, validation_data, test_dataset ), batch_size=BATCH_SIZES, device=device)

In [0]:
class RNN(nn.Module):
    def __init__(self, input_dims, embedded_dim, hidden_dim, output_dim):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_dims, embedded_dim)
        self.rnn = nn.RNN(embedded_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        return self.fc(hidden.squeeze(0))
        

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)


In [24]:
print('Number Of Trainable Parameters: {:,}'.format(sum(p.numel() for p in model.parameters())))

Number Of Trainable Parameters: 2,592,105


In [0]:
optimzer = optim.SGD(model.parameters(), lr=0.001)

In [0]:
loss = nn.BCEWithLogitsLoss()
model = model.to(device)
loss = loss.to(device)

In [0]:
def binary_accuracy(y_pred, y):
    rounded_pred = torch.round(torch.sigmoid(y_pred))
    correct = (rounded_pred == y).float()
    accuracy = correct.sum()/ len(y_pred)
    return accuracy
    

In [0]:
def train(model, iterator, optimizer, loss_function):
    epoch_accuracy = 0
    epoch_loss = 0
    
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        prediction = model(batch.text).squeeze(1)
        loss = loss_function(prediction, batch.label)
        accuracy = binary_accuracy(prediction, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_accuracy += accuracy.item()
        
    return epoch_loss / len(iterator) , epoch_acc / len(iterator)
        
        
        

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
def evaluate(model, iterator, loss_function):
    model.eval()
    
    epoch_loss = 0
    epoch_accuracy = 0
    
    with torch.no_grad():
        for batch in iterator:
            prediction = model(batch.text)
            loss = loss_function(predcition, batch.label)
            acc = binary_accuracy(prediction, batch.label)
            
            epoch_loss += loss.item()
            epoch_accuracy += acc.item()
        
    return epoch_loss / len(iterator) , epoch_accuracy/ len(iterator)


In [0]:
EPOCHS = 5

best_valid_loss = float('inf')

for epoch in EPOCHS:
    start_time = time.time()
    
    train_loss, train_accuracy = train(model, train_iterator, optimizer, loss)
    valid_loss, valid_accurayc = evaluate(model, valid_iterator, loss)
    
    end_time = time.time()
    
    epoch_mins, epoch_sec = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
        
    print('Epoch : {:2}, Epoch Time : {:2}m{:2}s'.format(epoch, epoch_mins, epoch_sec))
 